In [1]:
import uuid

In [2]:
import sqlite3

In [9]:
from datetime import datetime

In [3]:
import os

In [4]:
curr_sqlite_db_file = './output/2020-04/20120610_MCC/testdb.db'

In [36]:
with sqlite3.connect(curr_sqlite_db_file) as conn:
    c = conn.cursor()
    c.execute('''CREATE TABLE tracks (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, '''+
              '''                     track_uid TEXT NOT NULL UNIQUE, ''' + 
              '''                     start_dt TEXT, ''' +
              '''                     end_dt TEXT)''')
    c.execute('''CREATE TABLE track_labels (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, ''' + \
              '''                           label_uid TEXT NOT NULL UNIQUE, ''' + \
              '''                           track_id INTEGER NOT NULL, ''' + \
              '''                           prev_label_uid TEXT, ''' + \
              '''                           dt TEXT NOT NULL)''')
    conn.commit()

In [37]:
def insert_label(sqlite_db_fname, track_uid, label_uid, label_dt, prev_label_uid = ''):
    
    insert_track_query_text = '''INSERT OR IGNORE INTO tracks (track_uid) ''' + \
                              '''       VALUES ("%s")'''
    
    insert_label_query_text = '''INSERT INTO track_labels (label_uid, track_id, prev_label_uid, dt) ''' + \
                              '''       SELECT "%s", tr.id, "%s", "%s" FROM  tracks tr ''' + \
                              '''       WHERE tr.track_uid = "%s"'''
    
    alter_track_start_dt_query_text = '''UPDATE tracks SET start_dt = strftime("%%Y-%%m-%%dT%%H:%%M:%%S", (SELECT min(d1.dt) as dt_min FROM ''' + \
                                      '''    (SELECT datetime(dt) as dt FROM track_labels WHERE track_labels.track_id IN ''' + \
                                      '''        (SELECT id FROM tracks WHERE tracks.track_uid = "%s")''' + \
                                      '''    ) AS d1)) WHERE tracks.track_uid = "%s"'''
    
    alter_track_end_dt_query_text = '''UPDATE tracks SET end_dt = strftime("%%Y-%%m-%%dT%%H:%%M:%%S", (SELECT max(d1.dt) as dt_max FROM ''' + \
                                    '''    (SELECT datetime(dt) as dt FROM track_labels WHERE track_labels.track_id IN ''' + \
                                    '''        (SELECT id FROM tracks WHERE tracks.track_uid = "%s")''' + \
                                    '''    ) AS d1)) WHERE tracks.track_uid = "%s"'''
    
    with sqlite3.connect(sqlite_db_fname) as conn:
        c = conn.cursor()
        c.execute(insert_track_query_text % (track_uid))
        c.execute(insert_label_query_text % (label_uid,
                                             prev_label_uid,
                                             datetime.strftime(label_dt, DATETIME_FORMAT_STRING),
                                             track_uid))
        c.execute(alter_track_start_dt_query_text % (track_uid, track_uid))
        c.execute(alter_track_end_dt_query_text % (track_uid, track_uid))
        conn.commit()

In [38]:
for tr_i in range(5):
    tr_uid = str(uuid.uuid4()).replace('-', '')
    prev_label_uid = ''
    for l_i in range(10):
        label_uid = str(uuid.uuid4()).replace('-', '')
        insert_label(curr_sqlite_db_file,
                     tr_uid,
                     label_uid,
                     label_dt = datetime(2019, 7, 1, tr_i, l_i, 0),
                     prev_label_uid = prev_label_uid)
#         conn.commit()
        prev_label_uid = label_uid

In [8]:
# def get_track_labels(sqlite_db_fname, track_uid):
select_track_query_text = '''SELECT lab.label_uid, lab.prev_label_uid from track_labels lab ''' + \
        ''' INNER JOIN tracks tr ON tr.id = lab.track_id ''' + \
        ''' WHERE tr.track_uid = "%s" '''

In [9]:
conn = sqlite3.connect(curr_sqlite_db_file)

In [10]:
c = conn.cursor()

In [11]:
res = c.execute(select_track_query_text % '06688a01bec7485fa73d7ca5221217f1')

In [13]:
res_all = res.fetchall()

In [14]:
res_all

[('e7fb7c04d49c4bfeb593f36c761b4212', ''),
 ('92ad9b133f7c40b8957337f85a38e71a', 'e7fb7c04d49c4bfeb593f36c761b4212'),
 ('838376a670b7449ebf3ebb7ceb5f1346', '92ad9b133f7c40b8957337f85a38e71a'),
 ('6339b23dc10c413c83d9ba03ad664231', '838376a670b7449ebf3ebb7ceb5f1346'),
 ('180e69b958e144a5b5010d8a9ba6d253', '6339b23dc10c413c83d9ba03ad664231'),
 ('b9a7b97028634e74b9bd074fc9e5224f', '180e69b958e144a5b5010d8a9ba6d253'),
 ('1741b33c99544bf696f41438121a7c36', 'b9a7b97028634e74b9bd074fc9e5224f'),
 ('5c9ef4cb908f4920ad2aa768d9c42fa8', '1741b33c99544bf696f41438121a7c36'),
 ('50d279eae33a4548bd7a2b889c1a8c74', '5c9ef4cb908f4920ad2aa768d9c42fa8'),
 ('a09df7c3de0f4580b154b2fc31de0ffe', '50d279eae33a4548bd7a2b889c1a8c74')]

In [46]:
conn.close()